In [ ]:
# pip install ultralytics pillow lxml scikit-learn

# Import library
import os
import xml.etree.ElementTree as ET
from PIL import Image
import pandas as pd
from ultralytics import YOLO
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from PIL import Image
import shutil
from sklearn.model_selection import train_test_split


[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached lxml-5.3.0-cp312-cp312-win_amd64.whl.metadata (3.9 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 0.0/887.4 kB ? eta -:--:--
   ----- ---------------------------------- 112.6/887.4 kB 2.2 MB/s eta 0:00:01
   ------------ --------------------------- 266.2/887.4 kB 2.3 MB/s eta 0:00:01
   -------------------- ------------------- 460.8/887.4 kB 2.9 MB/s eta 0:00:01
   ---------------------------------------  880.6/887.4 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 887.4/887.4 kB 3.7 MB/s eta 0:00:00
Using cached lxml-5.3.0-cp312-cp312-win_amd64.whl (3.8 MB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)


In [ ]:
# Define class names and their corresponding IDs
class_names = {'with_mask': 0, 'without_mask': 1, 'mask_weared_incorrect': 2}
annotations_dir = 'datasets/annotations'
images_dir = 'datasets/images'
output_dir = 'datasets/dataset'
os.makedirs(f"{output_dir}/images", exist_ok=True)
os.makedirs(f"{output_dir}/labels", exist_ok=True)

In [ ]:
# Define paths
images_dir = 'datasets/images'
annotations_dir = 'datasets/annotations'

train_images_dir = 'datasets/train/images'
train_annotations_dir = 'datasets/train/annotations'
val_images_dir = 'datasets/val/images'
val_annotations_dir = 'datasets/val/annotations'
test_images_dir = 'datasets/test/images'
test_annotations_dir = 'datasets/test/annotations'

# Create directories if they don't exist
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_annotations_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_annotations_dir, exist_ok=True)
os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_annotations_dir, exist_ok=True)

# Get list of all image files
image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]

# Split the dataset into train and temp (which will be further split into val and test)
train_files, temp_files = train_test_split(image_files, test_size=0.3, random_state=42)

# Split the temp dataset into val and test
val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)

# Copy files to train directories
for file in train_files:
    shutil.copy(os.path.join(images_dir, file), os.path.join(train_images_dir, file))
    shutil.copy(os.path.join(annotations_dir, file.replace('.png', '.xml')), os.path.join(train_annotations_dir, file.replace('.png', '.xml')))

# Copy files to val directories
for file in val_files:
    shutil.copy(os.path.join(images_dir, file), os.path.join(val_images_dir, file))
    shutil.copy(os.path.join(annotations_dir, file.replace('.png', '.xml')), os.path.join(val_annotations_dir, file.replace('.png', '.xml')))

# Copy files to test directories
for file in test_files:
    shutil.copy(os.path.join(images_dir, file), os.path.join(test_images_dir, file))
    shutil.copy(os.path.join(annotations_dir, file.replace('.png', '.xml')), os.path.join(test_annotations_dir, file.replace('.png', '.xml')))

print("Dataset split into training, validation, and test sets.")

In [ ]:
yaml_text = """
train: datasets/train/images
test: datasets/test/images
val: datasets/train/images
nc: 3  # number of classes (with_mask, without_mask, mask_weared_incorrect)
names: ['with_mask', 'without_mask', 'mask_weared_incorrect']
"""

with open("data.yaml", 'w') as file:
    file.write(yaml_text)

with open("data.yaml") as f:
    contents = f.read()
    print(contents)

In [ ]:
# convert annotation to labels for yolo
def convert_xml_to_yolo(xml_file, output_dir, dtatype):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    image_id = os.path.splitext(os.path.basename(xml_file))[0]
    yolo_annotation = []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        if class_name not in class_names:
            print(f"Warning: Class '{class_name}' not found in class_names. Skipping.")
            continue
        class_id = class_names[class_name]
        bbox = obj.find('bndbox')
        xmin = float(bbox.find('xmin').text)
        ymin = float(bbox.find('ymin').text)
        xmax = float(bbox.find('xmax').text)
        ymax = float(bbox.find('ymax').text)
        
        x_center = (xmin + xmax) / 2.0
        y_center = (ymin + ymax) / 2.0
        width = xmax - xmin
        height = ymax - ymin
        
        # Load the image to get its dimensions
        img_path = os.path.join('datasets/' + dtatype + '/images', f"{image_id}.png")
        with Image.open(img_path) as img:
            img_width, img_height = img.size
        
        x_center /= img_width
        y_center /= img_height
        width /= img_width
        height /= img_height
        
        yolo_annotation.append(f"{class_id} {x_center} {y_center} {width} {height}")

    with open(os.path.join(output_dir, f"{image_id}.txt"), 'w') as f:
        f.write("\n".join(yolo_annotation))

# Convert all XML files in the annotations directory
data_list = ['train', 'test', 'val']
for i in data_list:
    annotations_dir = 'datasets/' + i + '/annotations'
    output_dir = 'datasets/' + i + '/labels'
    os.makedirs(output_dir, exist_ok=True)

    # Train conversion
    for xml_file in os.listdir(annotations_dir):
        if xml_file.endswith('.xml'):
            convert_xml_to_yolo(os.path.join(annotations_dir, xml_file), output_dir, i)

print("Conversion completed successfully.")

In [ ]:
# Load a YOLOv8 model
model = YOLO('yolov8n.pt')  # You can choose different versions like yolov8s.pt, yolov8m.pt, etc.

In [ ]:
# Fine-tune the model
model.train(data='data.yaml', epochs=1, imgsz=640, lr0=0.001)

# Save the trained model
model.save('trained_weights/yolov8n_trained.pt')

In [ ]:
# Validation of trained model weights
# Load the trained YOLOv8 model
model = YOLO('trained_weights/yolov8n_trained.pt')  # Make sure the path to your trained model file is correct

# Evaluate the model on the test data
results = model.val(data='data.yaml', split='test')  # Specify the test split
print(results)
# Print the evaluation results
print(f"mAP: {results.box.map}")
print(f"mAP@50: {results.box.map50}")
print(f"mAP@75: {results.box.map75}")
print(f"Precision: {results.box.mp}")
print(f"Recall: {results.box.mr}")

In [ ]:
# Load the trained YOLOv8 model
model = YOLO('trained_weights/yolov8n_trained.pt')  # Make sure the path to your trained model file is correct

# Get predictions from the model on the test data
results = model.predict(source='datasets/test/images', save=False)  # Specify the test images folder

# Define paths to the image and label folders
image_folder = 'datasets/test/images'  # Replace with the actual path to your test images folder
label_folder = 'datasets/test/labels'  # Replace with the actual path to your test labels folder

# Get image names from the folder
image_names = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]

# Initialize lists to store ground truth labels and predicted labels
y_true = []
y_pred = []

# Map labels from the folder and extract the first value before space as label
for image_name in image_names:
    label_file = os.path.join(label_folder, os.path.splitext(image_name)[0] + '.txt')
    if os.path.exists(label_file):
        with open(label_file, 'r') as f:
            label = f.read().strip().split()[0]  # Extract the first value before space as label
            y_true.append(int(label))  # Convert label to integer
    else:
        y_true.append(0)  # Default value if label file does not exist

for result in results:
    if result.boxes is not None:
        for pred in result.boxes.cls:  # Assuming result.boxes.cls gives the predicted classes
            y_pred.append(int(pred.item()))  # Convert tensor to integer
    else:
        y_pred.append(0)  # Default value if no prediction is made

# Ensure y_pred has the same length as y_true by padding with default values if necessary
while len(y_pred) < len(y_true):
    y_pred.append(0)

# Create a DataFrame with image names, actual labels, and predicted labels
output_df = pd.DataFrame({
    'Image Name': image_names,
    'Actual Label': y_true,
    'Predicted Label': y_pred
})

# Save the DataFrame to a CSV file
output_df.to_csv('output/custom_data_training.csv', index=False)

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix using seaborn heatmap with correct axis labels and title
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion  for face mask detection')
plt.savefig('output/confusion_matrix.png')
plt.show()

print("Output file and confusion matrix png saved successfully.")


# Export the trained model to ONNX format
model.export(format='onnx')
from ultralytics import YOLO
from PIL import Image

# Load the trained YOLOv8 model
model = YOLO('trained_weights/yolov8n_trained.onnx')  # Make sure the path to your model file is correct

# Function to perform inference on an image and save the predicted image
def perform_inference(image_path, save_path):
    # Load the image
    img = Image.open(image_path)
    
    # Perform inference
    results = model.predict(img)
    
    # Save the predicted image
    results[0].save(save_path)  # Access the first result and save the image with bounding boxes

    # Optionally, you can print the results
    predictions = results[0].boxes  # Access the bounding boxes
    print(predictions)
Image.open("output\confusion_matrix.png")

In [ ]:
# Example usage
image_path = 'datasets\predict\maksssksksss0.png'  # Replace with the path to your image
save_path = 'output\predicted\maksssksksss0.jpg'  # Replace with the path where you want to save the predicted image
perform_inference(image_path, save_path)

In [ ]:
# Export the trained model to ONNX format
model.export(format='onnx')

In [ ]:
# Load the trained YOLOv8 model
model = YOLO('trained_weights/yolov8n_trained.onnx')  # Make sure the path to your model file is correct

# Function to perform inference on an image and save the predicted image
def perform_inference(image_path, save_path):
    # Load the image
    img = Image.open(image_path)
    
    # Perform inference
    results = model.predict(img)
    
    # Save the predicted image
    results[0].save(save_path)  # Access the first result and save the image with bounding boxes

    # Optionally, you can print the results
    predictions = results[0].boxes  # Access the bounding boxes
    print(predictions)

# Example usage
image_path = 'datasets\predict\maksssksksss0.png'  # Replace with the path to your image
save_path = 'output\predicted\maksssksksss0.jpg'  # Replace with the path where you want to save the predicted image
perform_inference(image_path, save_path)